## Chương 11: Deep Learning in Image Processing - Object Detection, and more

#### __Bài tập 1:__ Hãy thử sử dụng mô hình YOLO v3 đã được huấn luyện sẵn để triển khai phát hiện đối tượng

#### __Giải thích code__

#### 1. Nhập các thư viện cần thiết

In [1]:
# cv2 để xử lý ảnh và video
import cv2
# numpy hỗ trợ tính toán số học, đặc biệt với các mảng đa chiều
import numpy as np
# time để đo thời gian, sử dụng trong việc tính FPS (Frames Per Second)
import time

#### 2. Tải và cấu hình mô hình YOLO v3

In [2]:
# Tải mạng YOLO v3 bằng cách sử dụng tệp trọng số (yolov3.weights) và tệp cấu hình (yolov3.cfg)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
# coco.names: Tệp chứa danh sách tên các lớp đối tượng mà YOLO v3 có thể phát hiện (ví dụ: người, xe hơi, chó, mèo,...)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
# Lấy tên tất cả các lớp trong mạng YOLO v3
layer_names = net.getLayerNames()
# Xác định các lớp xuất ra (output layers) của mô hình YOLO v3. Đây là các lớp mà mô hình sẽ trả về kết quả phát hiện đối tượng
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
# Tạo màu ngẫu nhiên cho mỗi lớp đối tượng để hiển thị khi vẽ khung hộp quanh đối tượng
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [3]:
# Mở webcam mặc định (thường là webcam tích hợp)
cap = cv2.VideoCapture(0)

# Chọn kiểu font để hiển thị văn bản trên ảnh
font = cv2.FONT_HERSHEY_PLAIN
# Biến để tính FPS (Frames Per Second)
starting_time = time.time()
frame_id = 0
# Bắt đầu vòng lặp vô hạn để xử lý liên tục các khung hình từ webcam
while True:
    # Đọc một khung hình từ webcam
    _, frame = cap.read()
    frame_id += 1
    # Lấy kích thước (chiều cao, chiều rộng, số kênh màu) của khung hình
    height, width, channels = frame.shape
    # Chuyển đổi khung hình thành blob để đưa vào mạng YOLO v3
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    # Đưa blob vào mạng YOLO v3
    net.setInput(blob)
    # Thực hiện truyền dữ liệu qua mạng và lấy kết quả từ các lớp xuất ra
    outs = net.forward(output_layers)
    # Lưu trữ ID lớp của đối tượng được phát hiện
    class_ids = []
    # Lưu trữ mức độ tự tin của dự đoán
    confidences = []
    # Lưu trữ tọa độ của khung hộp bao quanh đối tượng
    boxes = []
    # Duyệt qua từng lớp xuất ra
    for out in outs:
        # Duyệt qua từng phát hiện trong lớp đó
        for detection in out:
            # Lấy các điểm số cho từng lớp
            scores = detection[5:]
            # Xác định lớp có điểm số cao nhất
            class_id = np.argmax(scores)
            # Lấy điểm số tự tin cho lớp đó
            confidence = scores[class_id]
            # Chỉ xem xét những phát hiện có độ tự tin lớn hơn 0.2
            if confidence > 0.2:
                # center_x và center_y: Tọa độ của tâm khung hộp
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                # w và h: Chiều rộng và chiều cao của khung hộp
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # x và y: Tọa độ góc trên bên trái của khung hộp
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                # Thêm thông tin vào danh sách
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    # NMSBoxes: Giảm số lượng các khung hộp trùng lặp bằng cách giữ lại những khung hộp có độ tự tin cao nhất
    # Lưu trữ các chỉ mục của các khung hộp được giữ lại sau khi áp dụng NMS
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.3)

    for i in range(len(boxes)):
        # Kiểm tra xem chỉ mục hiện tại có nằm trong danh sách các khung hộp sau NMS hay không
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            # Vẽ khung hộp bao quanh đối tượng
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            # Hiển thị tên lớp và độ tự tin trên khung hộp
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 3, color, 3)

    # Thời gian đã trôi qua kể từ khi bắt đầu chạy vòng lặp
    elapsed_time = time.time() - starting_time
    # Tính số khung hình được xử lý mỗi giây
    fps = frame_id / elapsed_time
    # Hiển thị giá trị FPS trên khung hình tại vị trí (10, 50) với màu đen
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 4, (0, 0, 0), 3)
    # Hiển thị khung hình đã được xử lý với các khung hộp và văn bản
    cv2.imshow("Image", frame)
    # Chờ phím nhấn trong khoảng 1ms
    key = cv2.waitKey(1)
    # Nếu phím nhấn là Esc (mã ASCII: 27) thì thoát khỏi vòng lặp
    if key == 27:
        break

# Giải phóng tài nguyên của webcam
cap.release()
# Đóng tất cả các cửa sổ hiển thị của OpenCV
cv2.destroyAllWindows()